In [1]:
#%pip install scikit-surprise==1.1.0 # pickle
%pip install numpy==1.26.4
%pip install scikit-surprise



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [3]:
import pandas as pd
import pathlib

DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

C:\Users\USER\Downloads\Compressed\recommender-main\src\data


True

In [4]:
dataset = DATA_DIR / "ratings_small.csv"
dataset.exists()

True

In [5]:
df = pd.read_csv(dataset)
df["rating"].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [7]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)

In [8]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89366526, 0.90923224, 0.88804666, 0.90366779]),
 'test_mae': array([0.68804508, 0.7010151 , 0.68514674, 0.69572765]),
 'fit_time': (0.7218880653381348,
  0.7290875911712646,
  0.7302169799804688,
  0.828850269317627),
 'test_time': (0.13353729248046875,
  0.12312459945678711,
  0.13561272621154785,
  0.12237286567687988)}

In [9]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [10]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

RMSE: 0.6443


0.6443014499257635

In [11]:
sample_row = df.sample(n=1)
userId = sample_row["userId"].values[0]
movieId = sample_row["movieId"].values[0]
print(userId, movieId)

25 104


In [12]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.2873419477019152

In [13]:
import pickle

In [14]:
algo_data = {"model": algo}
with open("model.pkl", "wb") as f:
    pickle.dump(algo_data, f)

In [15]:
model_algo = None
with open("model.pkl", "rb") as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get("model")

In [16]:
model_algo.predict(uid=userId, iid=movieId).est

3.2873419477019152

In [17]:
sample_rows = df.sample(n=10).to_dict("records")
for row in sample_rows:
    userId = row["userId"]
    movieId = row["movieId"]
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

543 494 4.514040804246363
2 208 2.919714597688107
326 1374 3.464332238249954
388 2194 4.5578564797588825
17 4037 4.019330273426294
595 2324 3.8221958399716915
30 2916 3.7104182919546242
509 592 2.787597856795272
468 78574 3.2167448685027695
262 1073 3.0472712593229088
